In [14]:
import cv2
import time
import numpy as np
import os
from keras.models import load_model
from collections import Counter

drowsy_alert = 0  # Initialize drowsy alert counter

def preprocess_image(img, target_shape=(224, 224)):
    resized_img = cv2.resize(img, target_shape)
    return resized_img

def color_normalization(eye_region):
    lab = cv2.cvtColor(eye_region, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    filtered_eye = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return filtered_eye

def preprocess_eye_region(eye_region, target_shape=(224, 224)):
    resized_eye = cv2.resize(eye_region, target_shape)
    return resized_eye


def detect_pupil(eye_region, frame_counter):
    preprocessed_eye = preprocess_eye_region(eye_region)

    gray_eye = cv2.cvtColor(preprocessed_eye, cv2.COLOR_BGR2GRAY)

    _, thresholded = cv2.threshold(gray_eye, 75, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5, 5), np.uint8)
    thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(thresholded, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        valid_contours = [contour for contour in contours if cv2.contourArea(contour) > 75]

        if valid_contours:
            largest_contour = max(valid_contours, key=cv2.contourArea)

            x_pupil, y_pupil, w_pupil, h_pupil = cv2.boundingRect(largest_contour)

          
            if 10 < w_pupil < 50 and 10 < h_pupil < 50:
                cv2.rectangle(eye_region, (x_pupil, y_pupil), (x_pupil + w_pupil, y_pupil + h_pupil), (0, 255, 0), 2)

                new_frame_name = f"frame_{frame_counter}_pupil_detected.png"
                cv2.imwrite(os.path.join(output_folder_pupil, new_frame_name), cv2.cvtColor(eye_region, cv2.COLOR_BGR2RGB))
                return True

  
    new_frame_name = f"frame_{frame_counter}_no_pupil_detected.png"
    cv2.imwrite(os.path.join(output_folder_pupil, new_frame_name), cv2.cvtColor(eye_region, cv2.COLOR_BGR2RGB))
    return False


vgg_model_path = "VGG_RNR_model.h5"
vgg_model = load_model(vgg_model_path)

efficientnet_model_path = 'efficientnet_custom_c_n_model_MRL.h5'
efficientnet_model = load_model(efficientnet_model_path)

cap = cv2.VideoCapture(0)


total_personalization_time = 60  
batch_duration = 10
current_timestamp = time.time()
open_eye_counts = 0
closed_eye_counts = {}
start_frame_time = time.time()
prev_sleep_hours = int(input("Enter previous sleep hours: "))
working_hours = int(input("Enter working hours: "))
medication_taken = input("Have you taken any medications? (yes/no): ").lower() == 'yes'

if prev_sleep_hours >= 6 and 8 <= working_hours <= 12 and not medication_taken:
    sleep_duration = 0.50
else:
    sleep_duration = 0.30

start_total_time = time.time()
start_personalization_time = time.time()
monitoring = True
max_open_eye_count = 0
max_closed_eye_count = 0

# Personalization Phase
while time.time() - start_personalization_time < total_personalization_time:
    start_batch_time = time.time()

    open_eye_counts_batch = []
    closed_eye_counts_batch = []



    while time.time() - start_batch_time < batch_duration:
        ret, frame = cap.read()

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex, ey, ew, eh) in eyes:
                eye_region = frame[y+ey:y+ey+eh, x+ex:x+ex+ew]

                preprocessed_eye = preprocess_image(eye_region, target_shape=(224, 224))
                input_data_vgg = np.expand_dims(preprocessed_eye, axis=0)
                prediction_vgg = vgg_model.predict(input_data_vgg)
                final_prediction_vgg = 1 if prediction_vgg[0, 1] > 0.50 else 0

                # Define prediction_efficientnet before referencing it
                prediction_efficientnet = None

                # Process predictions with EfficientNet model based on VGG predictions
                if final_prediction_vgg == 0:
                    # Predict using EfficientNet model directly on preprocessed_eye
                    input_data_efficientnet = np.expand_dims(preprocessed_eye, axis=0)
                    prediction_efficientnet = efficientnet_model.predict(input_data_efficientnet)
                else: 
                    # If VGG predicts open eye, perform color normalization
                    filtered_eye = color_normalization(preprocessed_eye)
                    
                    # Predict using EfficientNet model on color normalized eye
                    input_data_efficientnet = np.expand_dims(filtered_eye, axis=0)
                    prediction_efficientnet = efficientnet_model.predict(input_data_efficientnet)

                # Handle predictions from EfficientNet model
                final_prediction_efficientnet = 1 if prediction_efficientnet[0, 1] > 0.5 else 0
                if final_prediction_efficientnet == 0:
                    # Closed eye prediction
                    closed_eye_counts_batch.append(1) # Update closed eye timestamp

                elif final_prediction_efficientnet == 1:
                    # Open eye prediction
                    open_eye_counts_batch.append(1) # Update open eye timestamp

    # Update max open and closed eye counts
    if open_eye_counts_batch:
        max_open_eye_count = max(max_open_eye_count, Counter(open_eye_counts_batch).most_common(1)[0][1])

    if closed_eye_counts_batch:
        max_closed_eye_count = max(max_closed_eye_count, Counter(closed_eye_counts_batch).most_common(1)[0][1])

    print(f"Personalization batch: Closed eye count: {len(closed_eye_counts_batch)}, Open eye count: {len(open_eye_counts_batch)}")
# Print max counts after all personalization batches are done
print(f"Max Open eye count: {max_open_eye_count}, Max Closed eye count: {max_closed_eye_count}")

time.sleep(sleep_duration)

if time.time() - start_total_time >= total_personalization_time:
    print("Personalization phase completed. Starting monitoring phase...")
# Create a folder to save drowsy frames
drowsy_frames_folder = 'drowsy_frames'
os.makedirs(drowsy_frames_folder, exist_ok=True)
# Monitoring Phase
frame_counter = 0 
while monitoring:
    ret, frame = cap.read()

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            eye_region = frame[y+ey:y+ey+eh, x+ex:x+ex+ew]

            preprocessed_eye = preprocess_image(eye_region, target_shape=(224, 224))
            input_data_vgg = np.expand_dims(preprocessed_eye, axis=0)
            prediction_vgg = vgg_model.predict(input_data_vgg)
            final_prediction_vgg = 1 if prediction_vgg[0, 1] > 0.50 else 0

            # Define prediction_efficientnet before referencing it
            prediction_efficientnet = None

            # Process predictions with EfficientNet model based on VGG predictions
            if final_prediction_vgg == 0:
                # Predict using EfficientNet model directly on preprocessed_eye
                input_data_efficientnet = np.expand_dims(preprocessed_eye, axis=0)
                prediction_efficientnet = efficientnet_model.predict(input_data_efficientnet)
            else: 
                # If VGG predicts open eye, perform color normalization
                filtered_eye = color_normalization(preprocessed_eye)
                
                # Predict using EfficientNet model on color normalized eye
                input_data_efficientnet = np.expand_dims(filtered_eye, axis=0)
                prediction_efficientnet = efficientnet_model.predict(input_data_efficientnet)

            # Handle predictions from EfficientNet model
            final_prediction_efficientnet = 1 if prediction_efficientnet[0, 1] > 0.5 else 0
            if final_prediction_efficientnet == 0:
                # Closed eye prediction
                closed_eye_counts_batch.append(time.time())# Update closed eye timestamp
    
                # Call pupil detection function
                pupil_detected = detect_pupil(eye_region, frame_counter)
    
                # Check if pupil is not detected
                if not pupil_detected:
                    print("Drowsy eye detected")
                    drowsy_alert = 1
                    print(f"drowsy_alert: {drowsy_alert}")
                    cv2.imwrite(os.path.join(drowsy_frames_folder, f"drowsy_frame_{frame_counter}.png"), frame)
                               
                    previous_frame_time = time.time() - batch_duration
                    previous_open_eye_count = sum(1 for t in open_eye_counts_batch if t >= previous_frame_time)
                    previous_closed_eye_count = sum(1 for t in closed_eye_counts_batch if t >= previous_frame_time)
                    
                    # Check if the previous closed eye count exceeds the maximum closed eye count
                    if previous_closed_eye_count > max_closed_eye_count:
                        print("Drowsy driving is noticed. Take a break!")
                        drowsy_alert += 1
                        print(f"drowsy_alert: {drowsy_alert}")
            elif final_prediction_efficientnet == 1:
                # Open eye prediction
                open_eye_counts_batch.append(time.time()) 
                # Check if open eye count is 2 times greater than max open eye count
                if len(open_eye_counts_batch) > 4 * max_open_eye_count and drowsy_alert == 0:
                    print("You are sleeping with open eyes. Please take a break!")
                    drowsy_alert += 1
                    print(f"drowsy_alert: {drowsy_alert}")
                    open_eye_counts_batch = []
                    
    
    
    key = cv2.waitKey(1)
    if key == 13:  # Enter key
        print("Monitoring stopped.")
        break
    
    time.sleep(sleep_duration)
# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



Enter previous sleep hours: 9
Enter working hours: 9
Have you taken any medications? (yes/no): NO
1/1 [==============================] - 0s 54ms/step
Personalization batch: Closed eye count: 0, Open eye count: 16
1/1 [==============================] - 0s 46ms/step
Personalization batch: Closed eye count: 0, Open eye count: 18
1/1 [==============================] - 0s 52ms/step
Personalization batch: Closed eye count: 0, Open eye count: 18
1/1 [==============================] - 0s 62ms/step
Personalization batch: Closed eye count: 0, Open eye count: 18
1/1 [==============================] - 0s 254ms/step


1/1 [==============================] - 0s 61ms/step
Personalization batch: Closed eye count: 0, Open eye count: 18
1/1 [==============================] - 0s 62ms/step
Personalization batch: Closed eye count: 0, Open eye count: 19
Max Open eye count: 19, Max Closed eye count: 0
Personalization phase completed. Starting monitoring phase...
1/1 [==============================] - 0s 51ms/step
Drowsy eye detected
drowsy_alert: 1
Drowsy driving is noticed. Take a break!
drowsy_alert: 2
1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 47ms/step
Drowsy eye detected
drowsy_alert: 1
Drowsy driving is noticed. Take a break!
drowsy_alert: 2
1/1 [==============================] - 0s 59ms/step
Drowsy eye detected
drowsy_alert: 1
Drowsy driving is noticed. Take a break!
drowsy_alert: 2
1/1 [==============================] - 0s 52ms/step
Drowsy eye detected
drowsy_alert: 1
Drowsy driving is noticed. Take a break!
drowsy_alert: 2
1/1 [==============================] - 0s 63ms/step
Drowsy eye detected
drowsy_alert: 1
Drowsy driving is noticed. Take a break!
drowsy_alert: 2
1/1 [==============================] - 0s 64ms/step
Drowsy eye detected
drowsy_alert: 1
Drowsy driving is noticed. Take a break!
drowsy_alert: 2
1/1 [==============================] - 0s 299ms/step


1/1 [==============================] - 0s 58ms/step


KeyboardInterrupt: 

IF 